In [1]:
using JuMP, Gurobi, MosekTools
using CSV
using DataFrames

In [2]:
df = CSV.read("julia_data_revised.csv", DataFrame)
print(df)

33×12 DataFrame
 Row │ item     Name                               Price    r_Lk     r_Uk     Bottle  Cans   IPA    Stout  Ale    Lager  Sour  
     │ String7  String                             Float64  Float64  Float64  Int64   Int64  Int64  Int64  Int64  Int64  Int64 
─────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ KO008    Kona Longboard Lager                   5.5  0.11427  0.0           0      1      0      0      0      1      0
   2 │ PB005    Pabst Blue Ribbon Lager (Can)          4.5  0.0342   0.0           0      1      0      0      0      1      0
   3 │ KO002    Kona Big Wave Golden Ale               5.5  0.08674  0.0           0      1      0      0      1      0      0
   4 │ 4P001    4 Pines Pacific Ale (Btl)              5.0  0.0734   0.0           1      0      0      0      1      0      0
   5 │ NB007    New Belgium Voodoo Ranger IPA          7.0  0.03587  0.07587       0      1 

In [3]:
p = df[!,"Price"]
ld = df[!, "r_Lk"]
ud = df[!, "r_Uk"]
ipa = df[!, "IPA"]
stout = df[!, "Stout"]
ale = df[!, "Ale"]
lager = df[!, "Lager"]
sour = df[!, "Sour"]
can = df[!,"Cans"];
btl = df[!, "Bottle"];

In [4]:
# Height of fridge
H = 53 # intended: 53

# Width of fridge slots
W = 16 # intended: 16

# Length of shelf
s_w = 8 # intended: 8

# Number of shelves
n_s = 8 # intended: 8

# Height of items
# [bottles cans]
h = [9 6] # intended: [9 6]

#offset
o = 0

# Number of items from the list to put into the model, it takes 1:types from the dataframe
types = 33; # intended 33

In [5]:
m = Model(Gurobi.Optimizer);

Set parameter Username
Academic license - for non-commercial use only - expires 2023-04-02


In [6]:
@variable(m, x[1:W, 1:(H-h[2]+1), 1+o:types+o], Bin)
@variable(m, d[1:W, 1:(H-h[2]+1), 1:2], Bin)
@variable(m, b[1+o:types+o], Bin)
@variable(m, s[1:2, 1:H], Bin)
@variable(m, u[1:W, 1:H, 0:1], Bin);

capacity constraints

In [7]:
# Volume creation constraint
for i in 1:W
    for j in 1:(H - h[2]+1)
        @constraint(m, sum(can[k]*x[i,j,k] for k in 1+o:types+o) - d[i,j,2] == 0)
        @constraint(m, sum(u[i,j+l,1] for l in 1:(h[2] - 1)) >= (h[2] - 1)*d[i,j,2])
    end
end

for i in 1:W
    for j in 1:(H - h[1]+1)
        @constraint(m, sum((1-can[k])*x[i,j,k] for k in 1+o:types+o) - d[i,j,1] == 0)
        @constraint(m, sum(u[i,j+l,1] for l in 1:(h[1] - 1)) >= (h[1] - 1)*d[i,j,1])
    end
end

#shelf constraint
for i in 1:2
    for j in 1:H
        @constraint(m, sum(u[((i-1)*s_w +1 +l) ,j ,0] for l in 0:(s_w-1)) == (s_w)*s[i,j])
    end
end

@constraint(m, sum(s) <= n_s) # Limit total number of shelves

# gravity constraint
for i in 1:W
    for k in 1+o:types+o
        if can[k] == 1
            for j in 1:(H - h[2])
                if j <= h[2]-1
                    @constraint(m, x[i,j+1,k] <= u[i,j,0])
                else
                    @constraint(m, x[i,j+1,k] <= u[i,j,0] + d[i,j-h[2]+1,2])
                end
            end
        else
            for j in 1:(H-h[1])
                @constraint(m, x[i,j+1,k] <= u[i,j,0])
            end
        end
    end
end

# Allow only 1 decision variable to be on at a spot
for i in 1:W
    for j in 1:H
        if j >= (H-h[2]+2)
            @constraint(m, sum(u[i,j,l] for l in 0:1) <= 1)
        else
            @constraint(m, sum(u[i,j,l] for l in 0:1) + sum(x[i,j,k] for k in 1+o:types+o) <= 1)
        end
    end
end

# Extra bottle variables
for i in 1:W
    for k in 1+o:types+o
        for j in 1:(H - h[2]+1)
            if can[k] == 0
                if j >= (H-h[1]+1)
                    @constraint(m, x[i,j,k] == 0)
                end
            end
        end
    end
end

# Stacking constraints
for i in 1:W
    for k in 1+o:types+o
        if can[k]==1
            for j in 1:(H - 3*h[2]+1)
                @constraint(m, x[i,(j+2*h[2]),k] <= 1 - d[i,j,2])
            end
        end
    end
end

Variety Constraints

In [8]:
for k in 1+o:types+o
    @constraint(m, sum(x[i,j,k] for i in 1:W for j in 1:(H-h[2]+1)) <= H*W*b[k])
    @constraint(m,  b[k] <= sum(x[i,j,k] for i in 1:W for j in 1:(H-h[2]+1)))
end

# Constraints on the amount of items from each category
@constraint(m, sum(ipa[c]*b[c] for c in 1:types) >= 5)
@constraint(m, sum(stout[c]*b[c] for c in 1:types) >= 2)
@constraint(m, sum(ale[c]*b[c] for c in 1:types) >= 5)
@constraint(m, sum(lager[c]*b[c] for c in 1:types) >= 1)
@constraint(m, sum(sour[c]*b[c] for c in 1:types) >= 2);

Demand Constraints

In [9]:
for k in 1+o:types+o
    @constraint(m,  ld[k]* sum(x) <= sum(x[i,j,k] for i in 1:W for j in 1:(H - h[2]+1))) # Lower Demand Constraint
    if ud[k] != 0
        @constraint(m,  ud[k]* sum(x) >= sum(x[i,j,k] for i in 1:W for j in 1:(H - h[2]+1))) # Upper Demand Constraint
    end
end

Objective Function

In [10]:
@objective(m, Max, sum(p[k]*sum(x[i,j,k] for i in 1:W for j in 1:(H - h[2]+1)) for k in 1+o:types+o));

In [11]:
optimize!(m)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 45088 rows, 28715 columns and 1444375 nonzeros
Model fingerprint: 0xb5cc89f6
Variable types: 0 continuous, 28715 integer (28715 binary)
Coefficient statistics:
  Matrix range     [8e-04, 8e+02]
  Objective range  [5e+00, 8e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 8e+00]
Presolve removed 21868 rows and 2016 columns
Presolve time: 4.64s
Presolved: 23220 rows, 26699 columns, 855053 nonzeros
Variable types: 0 continuous, 26699 integer (26655 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -0.0000000e+00   7.494000e+00   2.760182e+09      7s
     969    2.4187323e+03   0.000000e+00   0.000000e+00      7s
     969    2.4187323e+03   0.000000e+00   0.000000e+00      7s

Us

In [16]:
6*objective_value(m) # subtracts the weight constraint, added so that it prefers cans over bottles

4452.0

Interpreter

In [27]:
B = 1
U = 32
L = 1
R = 16
count_btl = 0
count_cans = 0

name = df[!, "Name"]

print("MAXIMIZED VALUE \n")
print(6*value.(sum(p[k]*sum(x[i,j,k] for i in 1:W for j in 1:(H - h[2]+1)) for k in 1+o:types+o)))
print(" SGD \n\n")

print("Type Information")
print("\n")

print("Lagers : ")
print(value.(sum(lager[k]*b[k] for k in 1+o:types+o)))
print("\n")
print("Ales   : ")
print(value.(sum(ale[k]*b[k] for k in 1+o:types+o)))
print("\n")
print("IPAs   : ")
print(value.(sum(ipa[k]*b[k] for k in 1+o:types+o)))
print("\n")
print("Stouts : ")
print(value.(sum(stout[k]*b[k] for k in 1+o:types+o)))
print("\n")
print("Sours  : ")
print(value.(sum(sour[k]*b[k] for k in 1+o:types+o)))
print("\n")
print("TOTAL  : ")
print(value.(sum(b)))
print("\n")
print("\n")
print("Item Breakdown")
print("\n")
for k in 1+o:types+o
    if value.(sum(x[:,:,k])) >= 1
#         print("item ")
#         print(k)
        print(name[k])
        print(" : ")
        print(6*value.(sum(x[:,:,k])))
        if btl[k] == 1
            print(" bottle")
            count_btl = count_btl + value.(sum(x[:,:,k]))
        else
            count_cans = count_cans + value.(sum(x[:,:,k]))
        end
        print("\n")
    end
end
print("\n")
print("bottles : ")
print(6*count_btl)
print("\n")
print("cans    : ")
print(6*count_cans)
print("\n")
print("TOTAL   : ")
print(6*value.(sum(x)))


print("\n")
print("\n")
print("SHELVES")
print("\n")
for i in 1:2
    for j in 1:H
        if value.(s[i,j]) == 1
            print("shelf at : (")
            print(i)
            print(", ")
            print(j)
            print(") \n")
        end
    end
end
print("TOTAL shelves : ")
print(value.(sum(s)))

print("\n")
print("\n")
print("Placement")
print("\n")
for j in B:U
    print(j )
    if j >= 10
        print(" ")
    else
        print("  ")
    end
end
print("\n")
for i in L:R
    for j in B:U
        if j > (H - h[2]+1)
            print("x")
        elseif value.(u[i,j,0]) == 1
            print("|")
        elseif value.(sum(btl[k]*x[i,j,k] for k in 1+o:types+o)) == 1
            print("b")
        elseif value.(sum(can[k]*x[i,j,k] for k in 1+o:types+o)) == 1
            print("c")
        else
            print("x")
        end
        print("  ")
        
    end
    print("\n")
end

MAXIMIZED VALUE 
4452.0 SGD 

Type Information
Lagers : 2.0
Ales   : 6.0
IPAs   : 12.0
Stouts : 2.0
Sours  : 2.0
TOTAL  : 24.0

Item Breakdown
Kona Longboard Lager : 84.0
Pabst Blue Ribbon Lager (Can) : 30.0
Kona Big Wave Golden Ale : 66.0
4 Pines Pacific Ale (Btl) : 54.0 bottle
New Belgium Voodoo Ranger IPA : 54.0
Colonial Pale Ale : 30.0
Colonial Draught Golden Ale : 30.0
Lost Coast Revenant IPA : 30.0 bottle
New Belgium Voodoo Ranger Juicy Hazy IPA : 36.0
Mother Earth Cali Creamin' Vanilla Cream Ale : 36.0
Lost Coast Hazy IPA : 12.0
Lost Coast Great White Wheat Ale (Can) : 72.0
Green Flash West Coast IPA : 30.0
Colonial Porter : 30.0
Green Flash Saturhaze Session Hazy IPA : 6.0
Green Flash Tropical DNA Hazy IPA : 24.0
Hawkers "Beneath The Trees We Gather" Yuzu & Plum Sour : 24.0
Lost Coast Sharkinator White IPA : 6.0 bottle
Lost Coast Peanut Butter Chocolate Milk Stout : 6.0 bottle
Green Flash Soul Style IPA : 6.0
Colonial IPA : 12.0
Kaiju Metamorphosis IPA : 18.0
Mother Earth Milk 

In [20]:
B = 33
U = 53
L = 1
R = 16
print("\n")
print("\n")
print("Placement")
print("\n")
for j in B:U
    print(j )
    if j >= 10
        print(" ")
    else
        print("  ")
    end
end
print("\n")
for i in L:R
    for j in B:U
        if j > (H - h[2]+1)
            print("x")
        elseif value.(u[i,j,0]) == 1
            print("|")
        elseif value.(sum(btl[k]*x[i,j,k] for k in 1+o:types+o)) == 1
            print("b")
        elseif value.(sum(can[k]*x[i,j,k] for k in 1+o:types+o)) == 1
            print("c")
        else
            print("x")
        end
        print("  ")
        
    end
    print("\n")
end



Placement
33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 
|  c  x  x  x  x  x  c  x  x  x  x  x  x  |  c  x  x  x  x  x  
|  c  x  x  x  x  x  c  x  x  x  x  x  x  |  c  x  x  x  x  x  
|  c  x  x  x  x  x  c  x  x  x  x  x  x  |  c  x  x  x  x  x  
|  c  x  x  x  x  x  c  x  x  x  x  x  x  |  c  x  x  x  x  x  
|  c  x  x  x  x  x  c  x  x  x  x  x  x  |  c  x  x  x  x  x  
|  c  x  x  x  x  x  c  x  x  x  x  x  x  |  c  x  x  x  x  x  
|  c  x  x  x  x  x  c  x  x  x  x  x  x  |  c  x  x  x  x  x  
|  c  x  x  x  x  x  c  x  x  x  x  x  x  |  c  x  x  x  x  x  
x  c  x  x  x  x  x  x  |  c  x  x  x  x  x  c  x  x  x  x  x  
x  c  x  x  x  x  x  x  |  c  x  x  x  x  x  c  x  x  x  x  x  
x  c  x  x  x  x  x  x  |  c  x  x  x  x  x  c  x  x  x  x  x  
x  c  x  x  x  x  x  x  |  c  x  x  x  x  x  c  x  x  x  x  x  
x  c  x  x  x  x  x  x  |  c  x  x  x  x  x  c  x  x  x  x  x  
x  c  x  x  x  x  x  x  |  c  x  x  x  x  x  c  x  x  x  x  x  
x  c  x  x  x  x  x  x  |  c

In [22]:
B = 1
U = 53
L = 1
R = 16
print("Check volume constraints") # Just to see if the constraints are being abided by
print("\n")
for j in B:U
    print(j )
    if j >= 10
        print(" ")
    else
        print("  ")
    end
end
print("\n")
for i in L:R
    for j in B:U
        if value.(u[i,j,0]) == 1
            print("|")
        elseif value.(u[i,j,1]) == 1
            print("b")
        else
            print("x")
        end
        print("  ")
    end
    print("\n")
end

Check volume constraints
1  2  3  4  5  6  7  8  9  10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 
x  b  b  b  b  b  b  b  b  |  x  b  b  b  b  b  x  b  b  b  b  b  |  x  b  b  b  b  b  b  b  b  |  x  b  b  b  b  b  x  b  b  b  b  b  b  |  x  b  b  b  b  b  
x  b  b  b  b  b  b  b  b  |  x  b  b  b  b  b  x  b  b  b  b  b  |  x  b  b  b  b  b  b  b  b  |  x  b  b  b  b  b  x  b  b  b  b  b  b  |  x  b  b  b  b  b  
x  b  b  b  b  b  b  b  b  |  x  b  b  b  b  b  x  b  b  b  b  b  |  x  b  b  b  b  b  b  b  b  |  x  b  b  b  b  b  x  b  b  b  b  b  b  |  x  b  b  b  b  b  
x  b  b  b  b  b  b  b  b  |  x  b  b  b  b  b  x  b  b  b  b  b  |  x  b  b  b  b  b  b  b  b  |  x  b  b  b  b  b  x  b  b  b  b  b  b  |  x  b  b  b  b  b  
x  b  b  b  b  b  b  b  b  |  x  b  b  b  b  b  x  b  b  b  b  b  |  x  b  b  b  b  b  b  b  b  |  x  b  b  b  b  b  x  b  b  b  b  b  b  |  x  b  b  b  b  b  
x  b  b  b  b  

In [23]:
print("Check capacity constraints") # Just to check where is 1 and where is 0 in the fridge
print("\n")
for j in B:U
    print(j )
    print(".0")
    if j >= 10
        print(" ")
    else
        print("  ")
    end
end
print("\n")

            
for i in L:R
    for j in B:U
        if j > H-h[2]+1
            print(value.(sum(u[i,j,k] for k in 0:1)))
        elseif j <= U
           print(value.(sum(u[i,j,k] for k in 0:1)+ sum(x[i,j,l] for l in 1+o:types+o)))
        else
            print(value.(sum(u[i,j,k] for k in 0:1)))
        end
        print("  ")
    end
    print("\n")
end
    
for i in L:R
    for j in B:U
        if j <= H - h[2]+1
           if value.(sum(u[i,j,k] for k in 0:1)+ sum(x[i,j,l] for l in 1+o:types+o)) >= 2
                    print("error at : (")
                    print(i)
                    print(", ")
                    print(j)
                    print(")")
                    print("\n")
            end
        else
            if value.(sum(u[i,j,k] for k in 0:1)) >= 2
                    print("error at : (")
                    print(i)
                    print(", ")
                    print(j)
                    print(")")
                    print("\n")
            end
        end
    end
end

Check capacity constraints
1.0  2.0  3.0  4.0  5.0  6.0  7.0  8.0  9.0  10.0 11.0 12.0 13.0 14.0 15.0 16.0 17.0 18.0 19.0 20.0 21.0 22.0 23.0 24.0 25.0 26.0 27.0 28.0 29.0 30.0 31.0 32.0 33.0 34.0 35.0 36.0 37.0 38.0 39.0 40.0 41.0 42.0 43.0 44.0 45.0 46.0 47.0 48.0 49.0 50.0 51.0 52.0 53.0 
1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  
1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  
1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  